In [315]:
using JuMP, Gurobi, Combinatorics, LightGraphs

# function solveMP()
const gurobi_env = Gurobi.Env()

function randPath(u,x,y,P_set, M_set, numPaths, numY)
    print("Start randPath \t ")
    global origin, destination
    Nset = []
    Aset = []
    simP = true
    u = origin
    push!(Nset, u)
    y_pos = findall(y.>0)
    M_pos = M_set[y_pos]
#     println("\tINSIDE randPath ==== ")
#     println("\ty_pos = ", y_pos)
#     println("\tM_pos = ", M_pos)
    while u != destination && simP == true
#         println("At node ", u)
        Q = 0
        outArcs = findall(arcs[:, 1].== u)
#         println("Outgoing arcs = ", outArcs)
        p_Val = zeros(length(outArcs))
        for i = 1:length(outArcs)
            a = outArcs[i]
            if x[a] == 1
#                 println("\t", a, " - interdicted" )
                p_Val[i] == 0
            else
                aM = findall((a in M_pos[m] for m = 1:length(M_pos)) .== true)
#                 println("\t", a, " - in Ms ", aM)
                if isempty(aM) == false
                    p_Val[i] = q[a]*sum(y[a] for a in aM)
                else 
                    p_Val[i] = 1
                end
            end
            Q = Q+p_Val[i]
        end
#         println("Q = ", Q)
#         println("p_Val = ", p_Val)
        if Q > 0
            p_Val = p_Val./Q
#             println("Normalized p_Val = ", p_Val)
            r = rand()
#             println("rnd = ", r)
            i = 0
            s_ = 0
            while s_ < r
                i = i+1
#                 println(" i = ", i)
                s_ = s_ + p_Val[i]
#                 println("s_ = ", s_)
            end
            a = outArcs[i]
#             println("Selected arc a = ", a)
            u = arcs[a, 2]
#             println("Selected arc ", arcs[a, :])
            if u in Nset
                simP = false
            end
            push!(Nset, u)
            push!(Aset, a)
        else
            simP = false
        end
    end
    println("Complete - randPath")
    return Aset, simP
end
function rowGen(theta_sol,x_sol,y_sol,P_set, M_set, numPaths, numY, newPath)
    print("Start rowGen \t")
    P, simP = randPath(theta_sol,x_sol,y_sol,P_set, M_set, numPaths, numY)
#     println("Begin rowGen simP = ", simP )
    q_P = ones(numY)
    if simP == true
        P_path = 0
#         println("P = ", P)
        for m = 1:numY
#             println("\tm = ", m)
            M = M_set[m]
            M_P = intersect(M, P)
            if length(M_P) > 0
                q_P[m] = prod((1-q[a]) for a in M_P)
            else
                q_P[m] = 1
            end
            P_path = P_path + q_P[m]*y_sol[m]
        end
        if  P_path > theta_sol && (P in P_set) == false
            newPath = true
        end
    end
    println("Complete - rowGen")
    return P, q_P, newPath
end
function randMonitor(u,x,y,P_set, M_set, numPaths, numY)
    print("Start randMonitor \t")
    Aset = []
    for i = 1:numPaths
        P = P_set[i]
        q_P = ones(numY)
        P_path = 0
        for m = 1:numY
            M = M_set[m]
            M_P = intersect(M, P)
            if length(M_P) > 0
                q_P[m] = prod((1-q[a]) for a in M_P)
            else
                q_P[m] = 1
            end
            P_path = P_path + q_P[m]*y_sol[m]
        end
        if  P_path -sum(x[a] for a in P) >= theta_sol 
            newPath = true
#             println("\tP_set = ", P_set)
#             println("\tAdd Path P = ", P_path)
            Aset = vcat(Aset, P)
        end
    end
    unique!(Aset)
#     println("Aset = ", Aset)
    M = []
    B = b_y
    

    Aset = intersect(findall(d_y.<= B), Aset)
    while isempty(Aset) == false
        q_Aset = q[Aset]
        Q = sum(q for q in q_Aset)
        q_Aset = q_Aset./Q
#         println("B_rem = ", B, "A feas = ", Aset)
        
        r = rand()
        i = 0
        s_ = 0
        while s_ < r
            i = i+1
            s_ = s_ + q_Aset[i]
        end
        a = Aset[i]
        push!(M, a)
        B = B - d_y[a]
#         println("selected ", a)
        deleteat!(Aset, i)
        deleteat!(q_Aset, i)
        Aset = intersect(findall(d_y.<= B), Aset)
#         Aset = intersect(findall(d_y.<= B), Aset)
#         if isempty(Aset) == false
#             q_Aset = q[Aset]
#             Q = sum(q for q in q_Aset)
#             q_Aset = q_Aset./Q
#         end
    end
    println("Complete  - randMonitor")
    return M
end

Academic license - for non-commercial use only - expires 2022-12-04


randMonitor (generic function with 1 method)

In [316]:
# setparams!(gurobi_env, OutputFlag = 0)

ins = 1#ARGS[1] #:10
dataSet = "N10_d20_"
println("Running...")
# global prob = "L" #"L"
#     include("./TestInstances/"*fileName*".jl")
global q = []
include("./"*dataSet*string(ins)*".jl")
global numArcs = length(arcs[:,1])
fileLoc = "C:/Users/din/Documents/GitHub/UncertainTarget/EnumX_RowCol2ndStage"
include(fileLoc*"/functionEnumFeasX.jl")
include(fileLoc*"/functionShortestPathBellmanFord.jl")# println(arcs)
# println(numArcs)
# global EnumY = EnumX(arcs, b_y, numArcs, d_y)
M = [findall(d_y.<=b_y)[1]]
global M_set = [M] #EnumY
println("M_set = ", M_set)
# push!(M_set,[19])
# println(M_set)
# break
global numY = length(M_set)
# println(EnumY)
# println("q = ", q)

path, gx = shortestPath_BellmanFord(q, arcs, numArcs, origin, destination)
arcsinPath = findall(path.>0)
println("P0 = ", arcs[arcsinPath,:])
println("\t", arcsinPath)
global P_set = [arcsinPath]
global numPaths = 1
global q_P1 = ones(numY)
global F_P1M = 1
global R = -sum(log(1-q_) for q_ in q)
# println("M_set = ", M_set)
for m = 1:numY
    M = M_set[m]
    
    M_P = intersect(M, arcsinPath)
    
#     println("M_P ", M_P)
    if length(M_P) > 0
#     q_P[m] = prod((1-q[a]) for a in M_P)
#     println("M = ", M)
#     print("q_M = ", q[M])
        q_P1[m] = prod((1-q[a]) for a in M_P)
    else
        q_P1[m] = 1
    end
#     println("M_set[m] = ", M_set[m])
#     println("F_P1M = ", q_P1[m])
end


# println(q_P1)
model = direct_model(Gurobi.Optimizer(gurobi_env))
#     set_optimizer_attribute(model, "Presolve", 0)
#     set_optimizer_attribute(model, "Precrush", 1)
#     set_optimizer_attribute(model, "DualReductions", 0)
#     set_optimizer_attribute(model, "PreQLinearize", 0)
#     set_optimizer_attribute(model, "Heuristics", 0)
#     set_optimizer_attribute(model, "MIPGap", 0)
# set_optimizer_attribute(model, "LazyConstraints", 1)
set_optimizer_attribute(model, "OutputFlag", 0)
# global loc = "./Documents/GitHub/UncertainTarget/EnumX_RowCol2ndStage/TestInstances/"
# println("Here")
# set_optimizer_attribute(model, "LogFile", loc*"L_"*dataSet*prob*"_"*string(ins)*".log")
# Presolve = 0,
#         PreCrush = 1,
#         Heuristics = 0,
cRefNum = 500 #max(500,3*length(P_set))
constr = Array{JuMP.ConstraintRef}(undef, cRefNum)
@variable(model, x[1:numArcs], Bin)
@variable(model, y[1:numY]>=0)
@variable(model, 1>= theta >= 0)#1e6)
@objective(model, Min, theta)
@constraint(model, sum(d_x[a]*x[a] for a=1:numArcs) <= b_x)
con0 = @constraint(model, sum(y[m] for m = 1:numY) == 1)
constr[1] = @constraint(model, theta >= sum(q_P1[m]*y[m] for m = 1:numY) - sum(x[a] for a in P_set[1]))
# for a in [5, 17, 18, 21]
#     @constraint(model, x[a] == 1)
# end
# @constraint(model, y[numY] == 1)
for iter = 1:5
#     println("A")
    numPaths = length(P_set)
    numY = length(M_set)
    optimize!(model)
    x_sol = JuMP.value.(x)
    y_sol = JuMP.value.(y)
    theta_sol = JuMP.value.(theta)
    c = zeros(numArcs)
    M_pos = findall(y_sol.>0)
    println("\n\niter = ", iter, ": theta = ", theta_sol)
    println("Interdict = \t", findall(x_sol.>0),"\t",arcs[findall(x_sol.>0),:])
    println("y_sol = \t", y_sol[M_pos], "\t", M_pos)
    println("\t\t ", M_set[M_pos])
    
    newPath = false
    for rep = 1:5
        P, q_P, newPath = rowGen(theta_sol,x_sol,y_sol,P_set, M_set, numPaths, numY, newPath)
        if newPath == true
            numPaths = numPaths + 1
            println("\tP_set = ", P_set)
            println("\tAdd Path P = ", P)
            push!(P_set, P)
            constr[numPaths] = @constraint(model, theta >= sum(q_P[m]*y[m] for m = 1:numY) - sum(x[a] for a in P))
        end
    end
    if newPath == false
        print("Start colGen")
        M = randMonitor(theta_sol,x_sol,y_sol,P_set, M_set, numPaths, numY)
        println("new M = ", M)
        if isempty(M) == false && (M in M_set) == false
            newM = true
            numY = numY +1
            push!(M_set, M)
            new_var = @variable(model, [numY], base_name = "y", lower_bound = 0)
            push!(y, new_var[numY])
            
            for i = 1:numPaths
                P = P_set[i]
                McapP = intersect(P, M)
                if length(McapP) > 0
                    q_P = prod((1-q[a]) for a in McapP)
#                     set_normalized_coefficient(constr[i], new_var[numY], 
#                         -(1+sum(log10(1-q[a]) for a in McapP) ) )
                else
                    q_P = 1 
                end
                set_normalized_coefficient(constr[i], new_var[numY], -q_P)
            end
            set_normalized_coefficient(con0, new_var[numY], 1)
        end
        println("Complete - colGen")
    end
end
x_sol = JuMP.value.(x)
y_sol = JuMP.value.(y)
theta_sol = JuMP.value.(theta)
println("\n\ntheta = ", theta_sol)
println("x_sol = ", x_sol)
M_pos = findall(y_sol.>0)
println("y_sol = \t", y_sol[M_pos])
println("\t ", M_set[M_pos])


Running...
M_set = Array{Int64,1}[[1]]
P0 = [1 8; 3 10; 5 3; 8 5]
	[2, 8, 12, 18]


iter = 1: theta = 0.0
Interdict = 	[8, 18]	[3 10; 8 5]
y_sol = 	[1.0]	[1]
		 Array{Int64,1}[[1]]
Start rowGen 	Start randPath 	 Complete - randPath
Complete - rowGen
Start rowGen 	Start randPath 	 Complete - randPath
Complete - rowGen
Start rowGen 	Start randPath 	 Complete - randPath
Complete - rowGen
Start rowGen 	Start randPath 	 Complete - randPath
Complete - rowGen
Start rowGen 	Start randPath 	 Complete - randPath
Complete - rowGen
Start colGenStart randMonitor 	Complete  - randMonitor
new M = Any[]
Complete - colGen


iter = 2: theta = 0.0
Interdict = 	[8, 18]	[3 10; 8 5]
y_sol = 	[1.0]	[1]
		 Array{Int64,1}[[1]]
Start rowGen 	Start randPath 	 Complete - randPath
Complete - rowGen
Start rowGen 	Start randPath 	 Complete - randPath
Complete - rowGen
	P_set = Array{Int64,1}[[2, 8, 12, 18]]
	Add Path P = Any[2, 19, 16]
Start rowGen 	Start randPath 	 Complete - randPath
Complete - rowGen
	P_set = Arr

In [312]:
# println("\n\ntheta = ", theta_sol)
# println("x_sol = ", x_sol)
# M_pos = findall(y_sol.>0)
# println("y_sol = \t", y_sol[M_pos])
# println("\t ", M_set[M_pos])
arcs[[5, 17, 18, 21],:]
arcs[[2, 20, 17, 15, 21, 13],:]
d_y[[2, 8, 12, 18, 19, 16, 15, 22, 17]]

9-element Array{Float64,1}:
  8.0
  2.0
  3.0
 10.0
  8.0
  9.0
  8.0
  4.0
  6.0

206-element Array{Any,1}:
 [4]                  
 [10]                 
 [1, 7]               
 [1, 20]              
 [2, 15]              
 [3, 14]              
 [5, 11]              
 [5, 12]              
 [6, 8]               
 [6, 9]               
 [6, 16]              
 [6, 19]              
 [7, 17]              
 ⋮                    
 [3, 9, 18, 19, 20]   
 [3, 15, 16, 18, 19]  
 [3, 16, 18, 19, 20]  
 [7, 8, 9, 16, 18]    
 [7, 8, 9, 18, 19]    
 [7, 8, 16, 18, 19]   
 [7, 9, 16, 18, 19]   
 [8, 9, 16, 18, 20]   
 [8, 9, 18, 19, 20]   
 [8, 16, 18, 19, 20]  
 [9, 16, 18, 19, 20]  
 [3, 8, 9, 16, 18, 19]

In [78]:
global EnumY = EnumX(arcs, b_y, numArcs, d_y)
global M_set = EnumY
a = 6

aM = findall((a in M_set[m] for m = 1:numY) .== true)
println(M_set[aM])

Any[[6, 8], [6, 9], [6, 16], [6, 19], [3, 6, 18]]


In [242]:
M1 = [1,2,4,2,3,4]
unique!(M1)
# a = [4]
# println(M1)
# intersect(M1, a)

4-element Array{Int64,1}:
 1
 2
 4
 3

In [243]:
println(M1)

[1, 2, 4, 3]


In [92]:
println(a)

[4]


In [244]:
M1 = intersect(M1, findall(d_y.<5))

3-element Array{Int64,1}:
 1
 4
 3

In [245]:
d_y

22-element Array{Float64,1}:
  4.0
  8.0
  4.0
  4.0
  3.0
  7.0
  3.0
  2.0
  2.0
  7.0
  6.0
  3.0
  2.0
  3.0
  8.0
  9.0
  6.0
 10.0
  8.0
  2.0
  6.0
  4.0

In [246]:
println(M1)

[1, 4, 3]


In [248]:
deleteat!(M1, 2)

2-element Array{Int64,1}:
 1
 3

In [269]:
M2= [2,3,5,6]

4-element Array{Int64,1}:
 2
 3
 5
 6

In [271]:
vcat(M1, M2)

6-element Array{Int64,1}:
 1
 3
 2
 3
 5
 6